In [6]:
import pandas as pd
import os
from tqdm import tqdm
import shutil

In [11]:
new_image_path = r'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated'
new_folder_names = [
    'no_contact',
    'tongue',
    'paw',
]
new_image_path

'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated'

In [12]:
existing_image_path = r'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'
existing_folder_names = [
    'train',
    'test',
    'validate',
]
existing_image_path

'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'

In [13]:
new_data = []
for folder_name in new_folder_names:
    df = pd.DataFrame({'filename':os.listdir(os.path.join(new_image_path,folder_name))})
    df['folder'] = folder_name
    new_data.append(df)
new_data = pd.concat(new_data)

In [14]:
new_data

,filename,folder
0,oeid_846487947__frame_121967.png,no_contact
1,oeid_846487947__frame_98778.png,no_contact
2,oeid_965930965__frame_22518.png,no_contact
3,oeid_826587940__frame_1770.png,no_contact
4,oeid_817267785__frame_125060.png,no_contact
...,...,...
63,oeid_813083478__frame_58575.png,paw
64,oeid_813083478__frame_58631.png,paw
65,oeid_977132456__frame_106747.png,paw
66,oeid_813083478__frame_58581.png,paw


In [15]:
existing_filenames = []
for foldername in existing_folder_names:
    for category in new_folder_names:
        existing_filenames += os.listdir(os.path.join(existing_image_path, foldername, category))

In [16]:
len(existing_filenames)

11316

In [17]:
new_data = new_data.sample(len(new_data)).reset_index()
new_data['set'] = None
new_data['label'] = None

In [18]:
def label(folder):
    if folder.startswith('tongue'):
        return 'tongue'
    elif folder.startswith('paw'):
        return 'paw'
    else:
        return 'no_contact'

In [19]:
new_data.loc[:0.8*len(new_data),'set'] = 'train'
new_data.loc[0.8*len(new_data):0.9*len(new_data),'set'] = 'validate'
new_data.loc[0.9*len(new_data):,'set'] = 'test'
new_data['label'] = new_data['folder'].map(label)

In [20]:
new_data

,index,filename,folder,set,label
0,442,oeid_846487947__frame_74720.png,no_contact,train,no_contact
1,1970,oeid_909090479__frame_135992.png,no_contact,train,no_contact
2,1182,oeid_933338703__frame_74437.png,tongue,train,tongue
3,2290,oeid_953443028__frame_10773.png,tongue,train,tongue
4,3928,oeid_846487947__frame_48150.png,no_contact,train,no_contact
...,...,...,...,...,...
8478,2489,oeid_823392290__frame_55740.png,tongue,test,tongue
8479,4156,oeid_842973730__frame_55326.png,no_contact,test,no_contact
8480,2897,oeid_962473308__frame_21343.png,tongue,test,tongue
8481,1744,oeid_850489605__frame_101608.png,tongue,test,tongue


In [21]:
new_data['label'].value_counts()

no_contact    4878
tongue        3537
paw             68
Name: label, dtype: int64

In [22]:
new_data['set'].value_counts()

train       6787
validate     848
test         848
Name: set, dtype: int64

In [23]:
data_dst = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'
for idx,row in tqdm(new_data.iterrows()):
    src = os.path.join(new_image_path,row['folder'],row['filename'])
    dst = os.path.join(existing_image_path,row['set'],str(row['label']))
    if os.path.exists(dst) == False:
        os.mkdir(dst)
    if row['filename'] not in existing_filenames:
        shutil.move(src,dst)
    else:
        print('did not move {}, already exists'.format(src))

2341it [00:29, 94.43it/s] 

did not move //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/paw/Thumbs.db, already exists


2448it [00:30, 96.35it/s] 

did not move //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/Thumbs.db, already exists


3585it [00:42, 102.60it/s]

...

3722it [00:43, 99.84it/s] 

did not move //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/tongue/oeid_896160394__frame_11440.png, already exists


4846it [00:54, 105.02it/s]

did not move //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/tongue/oeid_878358326__frame_56448.png, already exists


8483it [01:29, 95.17it/s] 


In [ ]:
target_path = '/ibs-dougo-ux1/root/local'
os.path.exists(target_path)

In [ ]:
from dirsync import sync
source_path = data_dst = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'
target_path = '/Give/Target/Folder/Here'

sync(source_path, target_path, 'sync') #for syncing one way
# sync(target_path, source_path, 'sync') #for syncing the opposite way